In [15]:
import pandas as pd
import re
import spacy


In [23]:
import nltk
import os

# Forcer un répertoire spécifique pour télécharger les données
nltk_path = os.path.join(os.environ['APPDATA'], 'nltk_data')

# Télécharger les ressources nécessaires directement dans ce dossier
nltk.download('punkt', download_dir=nltk_path)
nltk.download('stopwords', download_dir=nltk_path)
nltk.download('wordnet', download_dir=nltk_path)
nltk.download('omw-1.4', download_dir=nltk_path)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HellaBOUHADDA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HellaBOUHADDA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HellaBOUHADDA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HellaBOUHADDA\AppData\Roaming\nltk_data...


True

In [3]:
# Chargement du modèle spaCy
nlp = spacy.load("en_core_web_sm")

In [4]:
# Chargement du dataset brut
file_path = "data/dataset.csv"
df = pd.read_csv(file_path)
df['humor'] = df['humor'].astype(bool)

In [5]:
# Nettoyage de base
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    return text

df['text_clean'] = df['text'].apply(clean_text)

In [6]:
# Suppression des stopwords + lemmatisation
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


In [29]:
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess_text_spacy(text):
    doc = nlp(text)
    tokens = [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop and not token.is_punct and token.is_alpha
    ]
    return " ".join(tokens)


In [31]:
from tqdm.notebook import tqdm
tqdm.pandas()

df['text_processed'] = df['text_clean'].progress_apply(preprocess_text_spacy)


  0%|          | 0/200000 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

# Calcul de la longueur des textes (en nombre de mots)
df['text_length'] = df['text'].apply(lambda x: len(str(x).split()))


# Assure-toi que ce bloc est à exécuter à la fin du notebook EDA
assert 'text' in df.columns, "Colonne 'text' manquante."
assert 'humor' in df.columns, " Colonne 'humor' manquante."
assert pd.api.types.is_bool_dtype(df['humor']), "Colonne 'humor' n'est pas booléenne."
assert 'text_clean' in df.columns, "Colonne 'text_clean' manquante."
assert df['text_clean'].str.len().gt(0).all(), "Certaines lignes de 'text_clean' sont vides."
assert 'text_length' in df.columns, "Colonne 'text_length' manquante."
assert df['text_length'].gt(0).all(), "Certaines longueurs de texte sont nulles."

print(" Toutes les étapes principales du EDA sont validées avec succès ! 💯")


✅ Toutes les étapes principales du EDA sont validées avec succès ! 💯


In [35]:
# Sauvegarde du dataset nettoyé
output_path = "data/clean_dataset.csv"
df.to_csv(output_path, index=False)

print(f" Dataset nettoyé sauvegardé dans : {output_path}")


 Dataset nettoyé sauvegardé dans : data/clean_dataset.csv
